
Baseball Steroid Use
When the Mitchell Report on steroid use in Major League Baseball [MLB], was published people were surprised at both the number and names of those who were mentioned. The dataset was created from data found in the Mitchell Report. Baseball players are shown as green nodes. Those who were found to be providers of steroids and other illegal performance enhancing substances appear as red nodes. The links reveal the flow of chemicals -- from providers to players. See http://orgnet.com/steroids.html
File Downloads: baseball.zip
How to Cite: Mitchell, George J. (2007). Mitchell Report: Report to the Commissioner of Baseball of an Independent Investigation into the Illegal use of Steroids and Other Performance Enhancing Substances by Players in Major League Baseball. Retreived from: http://orgnet.com/steroids.html.

In [59]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
from zipfile import ZipFile
from io import BytesIO
import urllib.request
import csv

# Download and extract the baseball data
url = 'http://www.casos.cs.cmu.edu/tools/datasets/external/baseball/baseball.zip'
with urllib.request.urlopen(url) as stream:
    with ZipFile(BytesIO(stream.read())) as archive:
        df = pd.read_csv(archive.open('BASEBALL_1M.csv'), index_col=0)


# Create a new graph object
G = nx.Graph()

# Open the CSV file and iterate over the rows and columns
with open('BASEBALL_1M.csv') as f:
    reader = csv.reader(f)
    header = next(reader) # Read the header row
    for i, row in enumerate(reader):
        for j, val in enumerate(row):
            if j == 0:
                # Skip the first column
                continue
            if int(val) > 0:
                # Add an edge to the graph if the value is non-zero
                G.add_edge(row[0], header[j], weight=int(val))

# Print the edges and their weights
for edge in G.edges(data=True):
    print(edge)



('Victor_Conte', 'Victor_Conte', {'weight': 1})
('Victor_Conte', 'Jeremy_Giambi', {'weight': 1})
('Victor_Conte', 'Jason_Giambi', {'weight': 1})
('Victor_Conte', 'Gary_Sheffield', {'weight': 1})
('Victor_Conte', 'Barry_Bonds', {'weight': 1})
('Victor_Conte', 'Armando_Rios', {'weight': 1})
('Jeremy_Giambi', 'Jeremy_Giambi', {'weight': 1})
('Jeremy_Giambi', 'Jason_Giambi', {'weight': 1})
('Jeremy_Giambi', 'Gary_Sheffield', {'weight': 1})
('Jeremy_Giambi', 'Barry_Bonds', {'weight': 1})
('Jeremy_Giambi', 'Armando_Rios', {'weight': 1})
('Jason_Giambi', 'Jason_Giambi', {'weight': 2})
('Jason_Giambi', 'Gary_Sheffield', {'weight': 2})
('Jason_Giambi', 'Bobby_Estalella', {'weight': 1})
('Jason_Giambi', 'Benito_Santiago', {'weight': 1})
('Jason_Giambi', 'Marvin_Bernard', {'weight': 1})
('Jason_Giambi', 'Barry_Bonds', {'weight': 2})
('Jason_Giambi', 'Armando_Rios', {'weight': 2})
('Jason_Giambi', 'Randy_Velarde', {'weight': 1})
('Gary_Sheffield', 'Gary_Sheffield', {'weight': 2})
('Gary_Sheffield'